In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE149980"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE149980"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE149980.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE149980.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE149980.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of Lymphoblastoid Cell Lines–LCLs from Depressed Patients after in-vitro treatment with citalopram–CTP"
!Series_summary	"We used whole gene gene expression profiling to identify potential gene expression biomarkers associated for the treatment individualization of unipolar depression."
!Series_overall_design	"Gene expression was measured after 24 and 48 hours of in-vitro treatment with 3 µM CTP in n=17 LCLs derived from depressed patients with documented clinical treatment outcome to SSRIs."
Sample Characteristics Dictionary:
{0: ['response status: responder', 'response status: non-responder'], 1: ['tissue: Lymphoblastoid Cell Lines (LCLs)']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Dict, Any, Callable
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data of Lymphoblastoid Cell Lines
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the Sample Characteristics Dictionary:
# Key 0 contains 'response status' which can be used as our trait (depression treatment response)
# There's no age or gender information
trait_row = 0
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert depression treatment response to binary value."""
    if isinstance(value, str):
        # Extract the part after the colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        # Convert to binary
        if "responder" in value.lower() and "non" not in value.lower():
            return 1  # Responder
        elif "non-responder" in value.lower():
            return 0  # Non-responder
    
    return None  # For any other or unknown values

def convert_age(value: str) -> Optional[float]:
    """Convert age to float. Not used in this dataset."""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary. Not used in this dataset."""
    return None

# 3. Save Metadata
# Conduct initial filtering on dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We should use the clinical_data that was loaded in a previous step
    # Since the actual clinical_data should be available in the environment
    # We'll use geo_select_clinical_features with the existing clinical_data
    
    try:
        # Extract clinical features using the provided function
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Assuming clinical_data is already loaded
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted data
        preview = preview_df(selected_clinical_df)
        print("Preview of extracted clinical data:")
        print(preview)
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
    except NameError:
        print("Warning: clinical_data not found. The clinical data extraction step cannot be completed.")
        print("Please ensure the clinical_data DataFrame is available from a previous step.")


Preview of extracted clinical data:
{'GSM4519184': [1.0], 'GSM4519185': [1.0], 'GSM4519186': [1.0], 'GSM4519187': [1.0], 'GSM4519188': [1.0], 'GSM4519189': [1.0], 'GSM4519190': [1.0], 'GSM4519191': [1.0], 'GSM4519192': [1.0], 'GSM4519193': [1.0], 'GSM4519194': [1.0], 'GSM4519195': [1.0], 'GSM4519196': [1.0], 'GSM4519197': [1.0], 'GSM4519198': [1.0], 'GSM4519199': [1.0], 'GSM4519200': [1.0], 'GSM4519201': [1.0], 'GSM4519202': [0.0], 'GSM4519203': [0.0], 'GSM4519204': [0.0], 'GSM4519205': [0.0], 'GSM4519206': [0.0], 'GSM4519207': [0.0], 'GSM4519208': [0.0], 'GSM4519209': [0.0], 'GSM4519210': [0.0], 'GSM4519211': [0.0], 'GSM4519212': [0.0], 'GSM4519213': [0.0], 'GSM4519214': [0.0], 'GSM4519215': [0.0], 'GSM4519216': [0.0], 'GSM4519217': [0.0], 'GSM4519218': [1.0], 'GSM4519219': [1.0], 'GSM4519220': [1.0], 'GSM4519221': [1.0], 'GSM4519222': [1.0], 'GSM4519223': [1.0], 'GSM4519224': [1.0], 'GSM4519225': [1.0], 'GSM4519226': [1.0], 'GSM4519227': [1.0], 'GSM4519228': [1.0], 'GSM4519229': [1.0

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Depression/GSE149980/GSE149980_series_matrix.txt.gz


Gene data shape: (50739, 68)
First 20 gene/probe identifiers:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyze the gene identifiers
# These identifiers (like '(+)E1A_r60_1', 'A_19_P00315452') are not standard human gene symbols
# They appear to be probe IDs from a microarray platform that need to be mapped to gene symbols

# Human gene symbols would typically be like BRCA1, TP53, IL6, etc.
# The identifiers we see are platform-specific probe IDs that need mapping

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


Platform title found: Agilent-039494 SurePrint G3 Human GE v2 8x60K Microarray 039381 (Probe Name version)



Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220', 'A_33_P3236322', 'A_33_P3319925', 'A_21_P0000509', 'A_21_P0000744', 'A_24_P215804'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220', 'A_33_P3236322', 'A_33_P3319925', 'A_21_P0000509', 'A_21_P0000744', 'A_24_P215804'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466', nan, 'XM_001133269', 'NR_024244', 'NR_038269', 'NM_016951'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466', 'AK128005', 'XM_001133269', 'NR_024244', 'NR_038269', 'NM_016951'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0, 100129869.0, 730249.0, nan, nan, 51192.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3', 'LOC100129869', 'IRG1', 'SNAR-G2', 'LOC100506844', 'CKLF'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium vol

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation that map to gene identifiers and gene symbols
# Based on the preview of gene_annotation, 'ID' appears to be the gene identifier column, matching the identifiers seen in gene_data
# 'GENE_SYMBOL' contains the gene symbols we want to map to

# 2. Create the gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of the gene mapping dataframe:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene-level expression data
# Use the apply_gene_mapping function to perform the conversion
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Transformed gene expression data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# 4. Further normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalizing gene symbols, shape: {gene_data.shape}")
print("Preview of normalized gene expression data (first 5 genes, first 5 samples):")
print(gene_data.iloc[:5, :5])

# 5. Save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (46204, 2)
First few rows of the gene mapping dataframe:
              ID          Gene
2   A_23_P117082         HEBP1
3  A_33_P3246448         KCNE4
4  A_33_P3318220        BPIFA3
5  A_33_P3236322  LOC100129869
6  A_33_P3319925          IRG1


Transformed gene expression data shape: (20353, 68)
First few gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT',
       'A4GNT', 'AA06'],
      dtype='object', name='Gene')
After normalizing gene symbols, shape: (19847, 68)
Preview of normalized gene expression data (first 5 genes, first 5 samples):


          GSM4519184  GSM4519185  GSM4519186  GSM4519187  GSM4519188
Gene                                                                
A1BG        0.857412    0.183019   -1.851454   -1.841433    0.033260
A1BG-AS1    0.193574    0.635310   -0.273058    0.066432    0.151078
A1CF        0.951021   -0.351138   -0.663651   -0.789045    0.338096
A2M        -0.620814   -0.329174   -0.151582   -0.199618   -1.207462
A2ML1       0.823854   -0.336316   -0.476297   -0.466115    0.673593


Gene expression data saved to ../../output/preprocess/Depression/gene_data/GSE149980.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. We'll normalize gene symbols in the gene expression data
# Note: We've already done this in step 6, so we can skip this part

# 2. Link the clinical and genetic data
# First, let's make sure we have the correct clinical data from step 2
# Review the clinical data attributes from step 2
def convert_trait(value: str) -> int:
    """Convert depression treatment response to binary value."""
    if isinstance(value, str):
        # Extract the part after the colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        # Convert to binary
        if "responder" in value.lower() and "non" not in value.lower():
            return 1  # Responder
        elif "non-responder" in value.lower():
            return 0  # Non-responder
    
    return None  # For any other or unknown values

# Get clinical data using the correct row index identified in step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=0,  # Using row 0 for response status as identified in step 2
    convert_trait=convert_trait,
    age_row=None,  # No age data available
    convert_age=None,
    gender_row=None,  # No gender data available
    convert_gender=None
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from Lymphoblastoid Cell Lines of depressed patients with SSRI treatment outcomes (responders/non-responders)."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Selected clinical data shape: (1, 68)
Clinical data preview:
{'GSM4519184': [1.0], 'GSM4519185': [1.0], 'GSM4519186': [1.0], 'GSM4519187': [1.0], 'GSM4519188': [1.0], 'GSM4519189': [1.0], 'GSM4519190': [1.0], 'GSM4519191': [1.0], 'GSM4519192': [1.0], 'GSM4519193': [1.0], 'GSM4519194': [1.0], 'GSM4519195': [1.0], 'GSM4519196': [1.0], 'GSM4519197': [1.0], 'GSM4519198': [1.0], 'GSM4519199': [1.0], 'GSM4519200': [1.0], 'GSM4519201': [1.0], 'GSM4519202': [0.0], 'GSM4519203': [0.0], 'GSM4519204': [0.0], 'GSM4519205': [0.0], 'GSM4519206': [0.0], 'GSM4519207': [0.0], 'GSM4519208': [0.0], 'GSM4519209': [0.0], 'GSM4519210': [0.0], 'GSM4519211': [0.0], 'GSM4519212': [0.0], 'GSM4519213': [0.0], 'GSM4519214': [0.0], 'GSM4519215': [0.0], 'GSM4519216': [0.0], 'GSM4519217': [0.0], 'GSM4519218': [1.0], 'GSM4519219': [1.0], 'GSM4519220': [1.0], 'GSM4519221': [1.0], 'GSM4519222': [1.0], 'GSM4519223': [1.0], 'GSM4519224': [1.0], 'GSM4519225': [1.0], 'GSM4519226': [1.0], 'GSM4519227': [1.0], 'GSM4519228': 

Data shape after handling missing values: (68, 19848)
For the feature 'Depression', the least common label is '0.0' with 32 occurrences. This represents 47.06% of the dataset.
The distribution of the feature 'Depression' in this dataset is fine.



Linked data saved to ../../output/preprocess/Depression/GSE149980.csv
